# Experiment for Shearing Box Simulation

In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import seaborn as sns
import xarray
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import stats
from tqdm import tqdm

import sys
sys.path.insert(1, '/home/alam/Desktop/Code/jax-cfd/jax_cfd/')
import jax_cfd.sb as cfd
import utils

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
size = 64
density = 1.
viscosity = 4e-3

angular_velocity = 1.0
shear_rate = 3/2 * angular_velocity

aspect_ratio = 1.5 #2.0

total_time = 2 #2 * jnp.pi * 10 / (shear_rate * 2 * jnp.pi)
outer_steps = 200

max_velocity = shear_rate * 2 * jnp.pi
cfl_safety_factor = 0.0005

seed = 0
case = 'background shear' # also supports 'kida'

In [3]:
# Define the physical dimensions of the simulation.
domain = ((-jnp.pi, jnp.pi), (-jnp.pi, jnp.pi)) # ((0, 2*jnp.pi), (0, 2*jnp.pi)) #
grid = cfd.grids.Grid((size, size), domain=domain)

# Construct an initial velocity field with constant x velocity
x0 = 0 #0.5 * 2 * jnp.pi
y0 = 0 #0.5 * 2 * jnp.pi
sigma = 0.05 * 2 * jnp.pi

def smooth_mask(x, y, x0, y0, aspect_ratio, width=0.1):
    phi = ((x - x0)**2 / 1.0**2 + (y - y0)**2 / aspect_ratio**2) - 1.0
    return 0.5 * (1.0 - jnp.tanh(phi / width))

if case == 'kida':
    v0 = cfd.initial_conditions.kida_vortex(0.2,0.1,grid,shear_rate)
elif case == 'background shear':
    x_velocity_fn = lambda x, y: np.zeros_like(x) 
    y_velocity_fn = lambda x, y: - shear_rate * x
    velocity_bc = (cfd.boundaries.shearingbox_boundary_conditions(grid.ndim, shear_rate, 0.0, 'vx'),
                cfd.boundaries.shearingbox_boundary_conditions(grid.ndim, shear_rate, 0.0, 'vy'))
    v0 = cfd.initial_conditions.initial_velocity_field((x_velocity_fn, y_velocity_fn), grid, velocity_bc)
elif case == 'turbulence':
    v0 = cfd.initial_conditions.filtered_velocity_field(
    jax.random.PRNGKey(seed), grid, max_velocity, shear_rate, 0.0)
    x = grid.mesh(grid.cell_faces[1])[0]
    background_shear = [jnp.zeros_like(x), -shear_rate * x]
    velocity_bc = (cfd.boundaries.shearingbox_boundary_conditions(grid.ndim, shear_rate, 0.0, 'vx'),
                cfd.boundaries.shearingbox_boundary_conditions(grid.ndim, shear_rate, 0.0, 'vy'))
    v0 = tuple([cfd.grids.GridVariable(cfd.grids.GridArray(u0.data + bs, u0.offset, grid), bc) for u0, bs, bc in zip(v0, background_shear, velocity_bc)])
else:
    raise ValueError('Unknown case')

In [ ]:
v0

In [5]:
def central_diff(v, dx):
    """
    Compute the central difference of a 2 dimensional velocity field.
    """
    vx = v[...,0]
    vy = v[...,1]
    return (np.roll(vx, -1, axis=0) - np.roll(vx, 1, axis=0)) / (2 * dx), \
           (np.roll(vx, -1, axis=1) - np.roll(vx, 1, axis=1)) / (2 * dx), \
           (np.roll(vy, -1, axis=0) - np.roll(vy, 1, axis=0)) / (2 * dx), \
           (np.roll(vy, -1, axis=1) - np.roll(vy, 1, axis=1)) / (2 * dx)


def compute_vorticity(v, dx):
    """
    Compute the vorticity of a 2D velocity field.
    """

    dvx_dx, dvx_dy, dvy_dx, dvy_dy = central_diff(v, dx)
    return dvy_dx - dvx_dy

x = np.linspace(0, 2 * np.pi, size, endpoint=False) + grid.step[0]/2  
#u0y = np.tile(shear_rate * x, (size, 1)).transpose()

v0_data = np.stack([v0[0].data, v0[1].data], axis = -1)
w0 = compute_vorticity(v0_data, grid.step[0])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12,4))
v0x_plot = ax[0].imshow(v0[0].data.transpose(), origin='lower', cmap=sns.cm.icefire, extent=(0,2*jnp.pi,0,2*jnp.pi))# vmin=-10, vmax=10, extent=(0,2*jnp.pi,0,2*jnp.pi))
v0y_plot = ax[1].imshow(v0[1].data.transpose(), origin='lower', cmap=sns.cm.icefire, vmin=-10, vmax=10, extent=(0,2*jnp.pi,0,2*jnp.pi))
w0_plot = ax[2].imshow(w0.transpose(), origin='lower', cmap=sns.cm.icefire, vmin=-10, vmax=10, extent=(0,2*jnp.pi,0,2*jnp.pi))
plt.colorbar(v0x_plot, ax=ax[0])
plt.colorbar(v0y_plot, ax=ax[1])
plt.colorbar(w0_plot, ax=ax[2])
ax[0].set_title(r"$v_{0,x}$")
ax[1].set_title(r"$v_{0,y}$")
ax[2].set_title(r"$w_{0}$")

In [ ]:
# Choose a time step.
dt = cfd.equations.stable_time_step(
    max_velocity, cfl_safety_factor, viscosity, grid)

inner_steps = total_time // (outer_steps*dt)

# Add in forcing
forcing = cfd.forcings.shearingbox_turbulence_forcing(grid, 
                                                 constant_magnitude=1.0,
                                                 constant_wavenumber=4.0,
                                                 linear_coefficient=-0.1,
                                                 angular_velocity=angular_velocity,
                                                 shear_rate=shear_rate,
                                                 dt=dt,
                                                 forced=False)

# In case for a change in viscosity term
diffuse = cfd.diffusion.diffuse

step_fn = jax.jit(cfd.equations.semi_implicit_navier_stokes_sb(
        density=density, viscosity=viscosity, dt=dt, grid=grid,
        diffuse=diffuse, forcing=forcing))

step_fn = cfd.funcutils.repeated_sb(step_fn,steps=inner_steps,dt=dt)
rollout_fn = jax.jit(cfd.funcutils.trajectory_sb(step_fn, outer_steps, Dt=dt*inner_steps))

print(dt)

In [ ]:
%time _, trajectory = jax.device_get(rollout_fn(v0))

In [9]:
ds = xarray.Dataset(
    {
        'u': (('time', 'x', 'y'), trajectory[0].data),
        'v': (('time', 'x', 'y'), trajectory[1].data),
    },
    coords={
        'x': grid.axes()[0],
        'y': grid.axes()[1],
        'time': dt * inner_steps * np.arange(outer_steps)
    }
)

In [ ]:
ds

In [ ]:
grid.cell_faces

In [ ]:
def vorticity(ds):
  return (ds.v.differentiate('x') - ds.u.differentiate('y')).rename('vorticity')

(ds.pipe(vorticity).thin(time=20).transpose()
 .plot.imshow(col='time', cmap=sns.cm.icefire, robust=True, col_wrap=5));

In [13]:
vorticity_numpy = np.transpose(ds.pipe(vorticity).to_numpy(),(0,2,1))

In [ ]:
fig, ax = plt.subplots()
vort = ax.imshow(vorticity_numpy[0], origin='lower', cmap=sns.cm.icefire, extent=[-jnp.pi, jnp.pi, -jnp.pi, jnp.pi])#, vmin=-5, vmax=5)
cbar = fig.colorbar(vort, ax=ax)
title = ax.set_title(f"Time = {0:.3f} s")

def update(frame):
    vort.set_data(vorticity_numpy[frame])
    title.set_text(f"Time = {ds.time[frame]:.3f} s")
    return [vort, title]

anim = animation.FuncAnimation(fig=fig, func=update, frames=int(vorticity_numpy.shape[0]), interval=30)
anim.save("sb_vorticity.gif", writer="pillow", fps=60)

In [ ]:
vx = trajectory[0].data
vy = trajectory[1].data

time = ds.time.data
energy = 1/2 * jnp.mean(vx**2 + vy**2, axis=(1,2))
plt.plot(time, energy)